## Custom Packages

In [1]:
import os, sys

processing_source_path = os.path.abspath('./../../../Processing/')
if(processing_source_path not in sys.path):
    sys.path.append(processing_source_path)
from processing import preprocessing
from processing import posprocessing
from processing import FeaturesExtract

Utils_source_path = os.path.abspath('./../../../Utils/')
if(Utils_source_path not in sys.path):
    sys.path.append(Utils_source_path)
from log_soak_test import soak_test

Embedded_source_path = os.path.abspath('./../../../Quantization/functions/')
if(Embedded_source_path not in sys.path):
    sys.path.append(Embedded_source_path)
from Embedded_Model import Embedded_Model

## Python Packages

In [2]:
#protocol communication liraries
import zmq
import logging
LOG_LEVEL=logging.DEBUG #change to logging.DEBUG to enable print logs
ZEROMQ_SOCKET="tcp://127.0.0.1:12345"

import numpy as np
import time
import pickle
import matplotlib.pyplot as plt

##  EDGE AI Deep learning model

#### Signal Processing functions

In [3]:
#  EDGE AI pre-definitions
pp = preprocessing(axis=1)
ppos=posprocessing()

#### Params and initial conditions

In [4]:
params = {
    'dataset_path': '/root/data/data/OpenBMI/',
    'channels': ['O1', 'Oz', 'O2', 'PO10', 'PO9', 'POz', 'PO3', 'PO4'],
    'freqs': [0, 12.0, 8.57, 6.67, 5.45],
    'num_harmonics': 3,
    'extracted_frequencies': [2,58],
    'sfreq': 1000,
    'features_type': 'psd',
    'preprocessing': 'log10',
    'num_channels': 8,
    'num_classes':  5,
    'num_samples': 16,
    'window_size': 1000,
    'step': 64*2,
    'num_voting': 3,
    'model_name': 'Conv2D_DW_GAP',
    'norm_axis': 3
}

window_EEG= np.random.uniform(0.00001, 0,size=(params['num_channels'],params['window_size']))
prep_time=[]
AI_time=[]
y_AI_array = np.zeros(params['num_voting'])

psd_2D = np.zeros((params['num_samples'],
                           params['extracted_frequencies'][1]-params['extracted_frequencies'][0],
                           params['num_channels']),dtype=float)

#### Load the ML model

In [5]:
AI_tf_model=Embedded_Model('./../../keras_quantized_model/Conv2D_DW_GAP')

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


#### Soak test log settings 

#### Communication protocol settings

In [6]:
context = zmq.Context()
subscriber = context.socket(zmq.SUB)
subscriber.connect(ZEROMQ_SOCKET)
subscriber.setsockopt(zmq.SUBSCRIBE, b"")

### Main-loop 

#### Soakt test time settings


In [7]:
#Acc variables
N_correct=0
N_interactions=0

# log updates time
log_update_time= 20 #seconds

# soak test total time
duration_time= 6 # hour
end_time= time.time()+ 60 * 60 * duration_time

update_time_duration=time.time()

In [8]:
log_name=f'logs/system_monitor_deep_embedded_model_during_{duration_time}_hours'
ST=soak_test(log_name=log_name)

#### AI model in loop

#### custom funcitons to 2D deep model 

In [10]:
def buffer_psd(spectra, x_vector):
    spectra = np.roll(spectra, -1, axis=0)
    spectra[-1,:,:] = x_vector
    
    return spectra

def calculate_psd(data, params):
    # Obs: the best
    # print('fft inputs', eeg.shape,rep,ch,nsteps,WinLength)
    
    myamp = np.array([])

    (f, S)= scipy.signal.welch(data, params['sfreq'], nperseg=None, window = 'box', scaling='spectrum', nfft=int(params['sfreq']), average='median')

    amp = np.abs(S)
    amp = np.log10(amp[:,params['extracted_frequencies'][0]:params['extracted_frequencies'][1]] + sys.float_info.epsilon)
        
    return amp 

def calculate_psd(data,params):
    
    FourierCoeff = np.fft.fft(data)/data.shape[0]
    amp = FourierCoeff[params['extracted_frequencies'][0]:params['extracted_frequencies'][1]]
    amp = 2*np.abs(amp)
        
    return np.log10(amp)

def calculate_amplitude2(data,params):
    
    # print('fft inputs', eeg.shape,rep,ch,nsteps,WinLength)
    
    myamp = np.array([])

    FourierCoeff = np.fft.fft(data)/params['window_size']
    # print('fourier',FourierCoeff.shape)
        
    DC = [np.abs(FourierCoeff[0])] # DC component
    amp = np.concatenate((DC, 2*np.abs(FourierCoeff[1:len(FourierCoeff)//2])))
    amp = np.log10(amp)
        
    return amp[params['extracted_frequencies'][0]:params['extracted_frequencies'][1]]

In [ ]:
def buffer_data(data, new_batch):
    data = np.roll(data,-new_batch.shape[0],axis=0)
    data[-new_batch.shape[0]:,:] = new_batch
    
    return data

def buffer_psd(spectra, x_vector):
    spectra = np.roll(spectra, -1, axis=0)
    spectra[-1,:,:] = x_vector
    
    return spectra

# calculate psd
def calculate_psd(data, params):
    
    # print('fft inputs', eeg.shape,rep,ch,nsteps,WinLength)
    
    myamp = np.array([])

    (f, S)= scipy.signal.welch(data, params['sfreq'], nperseg=None, window = 'box', scaling='spectrum', nfft=int(params['sfreq']), average='median')

    amp = np.abs(S)
    amp = np.log10(amp[ params['extracted_frequencies'][0]:params['extracted_frequencies'][1]] + sys.float_info.epsilon)
        
    return amp


#### Loop

In [ ]:
#while True:
Data=[]
all_labels=[]
all_pred=[]
while time.time() < end_time:
    data = subscriber.recv()
    # Convert received data in bit(4-bytes) to numpy array
    Data_array = np.frombuffer(data, dtype=np.float64).reshape(-1,params['num_channels']+1).T.astype(np.float32)
    
    label= Data_array[-1,:].astype(np.float32)

    Data += [Data_array[:-1,:]]
    
    if len(Data)>= params['step']//64:
        #Data_array=np.expand_dims(Data_array[:-1,:],0)
       
        window_EEG=pp.moving_window(window_EEG,np.hstack(Data),params['step'])   
        Data=[] # reset Data 

        tic=time.time()
        #preprcessing
    
        #psd_data=calculate_psd(window_EEG,params).transpose(1,0)
        
        psd_data=np.zeros([56,8])
        for ch in range(params['num_channels']):
            psd_data[:,ch]=calculate_amplitude2(window_EEG[ch,:],params)

        #psd_data.shape
        psd_2D = buffer_psd(psd_2D, psd_data)
        psd_2D_aux=np.expand_dims(psd_2D.transpose(2,0,1),0)

        norm_data=pp.NormMinMax_2D(psd_2D_aux,params['norm_axis'],0,1).transpose(0, 3, 2, 1)
        
        prep_time+=[time.time()-tic]
        

        #classification
        tic_model=time.time()

        y_AI_array = np.roll(y_AI_array,-1)

        #model_pred=AI_tf_model.classify_data(np.expand_dims(psd_2D.transpose(1,0,2),0))
        model_pred=AI_tf_model.classify_data(norm_data)

        if np.argmax(model_pred[0]) == 1 and np.argmax(model_pred[1]) != 0 :
        #if np.argmax(model_pred[1]) != 0:
    
            y_AI_array[-1] = np.argmax(model_pred[1])

            AI_time+=[time.time()-tic_model]

            #pos processing 
            y_votting=ppos.majority_voting(y_AI_array, params['num_voting'])
            all_pred+=[y_votting]
            
            all_pred+=[y_AI_array[-1]]
            all_labels+=[label[-1]]
                                           
            # Acc  
            if (y_votting == label[-1]):
                N_correct=N_correct+1
            
            N_interactions=N_interactions+1

    #log results
    if time.time() - update_time_duration>= log_update_time:
        if N_interactions!= 0:

            #reset log time 
            update_time_duration=time.time()
            #comput acc
            Acc=N_correct/N_interactions
            # update params60*
            print(f'Preprocessing Latency: {round(np.mean(prep_time)*1000,4)}ms | Model Predict Latency: {round(np.mean(AI_time)*1000,4)}ms | Model  accuracy: {Acc}')

            ST.set_model_performance(np.mean(prep_time), np.mean(AI_time),Acc)
            AI_time=[]
            prep_time=[]
            #uncomment to save logs and display it 
            #ST.log_info()
            
            N_correct=0
            N_interactions=0
        
print("Média de tempo no pré-processamento",np.array(prep_time).mean() *1000,'ms')
print("Máximo tempo no pré-processamento",np.array(prep_time).max() *1000,'ms')

print("Média de tempo do modelo AI",np.array(AI_time)[10:].mean() *1000,'ms')
print("Máximo tempo do modelo AI",np.array(AI_time)[10:].max() *1000,'ms')

subscriber.close()
context.term()

Preprocessing Latency: 0.6361ms | Model Predict Latency: 8.178ms | Model  accuracy: 0.6013071895424836
Preprocessing Latency: 0.669ms | Model Predict Latency: 8.5313ms | Model  accuracy: 0.7508090614886731
Preprocessing Latency: 0.6927ms | Model Predict Latency: 8.7017ms | Model  accuracy: 0.6925465838509317
Preprocessing Latency: 0.6502ms | Model Predict Latency: 8.5142ms | Model  accuracy: 0.7165109034267912
Preprocessing Latency: 0.6184ms | Model Predict Latency: 8.1117ms | Model  accuracy: 0.7046153846153846
Preprocessing Latency: 0.6046ms | Model Predict Latency: 8.0209ms | Model  accuracy: 0.72
Preprocessing Latency: 0.6343ms | Model Predict Latency: 8.0248ms | Model  accuracy: 0.809375
Preprocessing Latency: 0.5904ms | Model Predict Latency: 7.9708ms | Model  accuracy: 0.6818181818181818
Preprocessing Latency: 0.6301ms | Model Predict Latency: 7.9763ms | Model  accuracy: 0.6644736842105263
Preprocessing Latency: 0.7411ms | Model Predict Latency: 8.7149ms | Model  accuracy: 0.736